# Yeni başlayanlar için Çekiştirmeli Üretici Ağlar (ÇÜAlar) veya Generative Adversarial Networks (GANs) 
## Cifar 10 kullanarak renkli küçük resimler oluşturmak için GAN
###  [Meltem Atay](https://github.com/neuroscitechie) tarafından Deep Learning Türkiye  Ankara Toplantısı için Türkçeleştirildi.

<img src="notebook-images/dltr.png" />
Kaynak kitap: [the O'Reilly interactive tutorial on generative adversarial networks](https://www.oreilly.com/learning/generative-adversarial-networks-for-beginners). 

### Kullanılan kütüphaneler:

 [TensorFlow](https://www.tensorflow.org/install/), 
 [NumPy](https://docs.scipy.org/doc/numpy/user/install.html), 
 [matplotlib](https://matplotlib.org/) 
 [Jupyter](https://jupyter.readthedocs.io/en/latest/install.html)

## Giriş

Yann Lecun'a göre çekişmeli öğrenme derin öğrenmenin en iyi fikirlerindendir. 
<img src="notebook-images/lecun.jpeg" />
GAN'lar ilk olarak [Ian Goodfellow et al. in 2014](https://arxiv.org/abs/1406.2661) tarafından tarif edildiğinden beri büyük ilgi oldağı olmuştur.
GAN'lar girdi olarak verilen verisetinden sentetik veri seti oluşturmaya yönelik çalışan yapay sinir ağlarıdır.
Örnek GAN çalışmaları:
<img src="notebook-images/lsun_bedrooms_real.png" />

<img src="notebook-images/lsun_bedrooms_five_epoch_samples.png" />

<img src="notebook-images/albums_128px.png" />

<img src="notebook-images/faces_arithmetic_collage_v2.png" />

<img src="notebook-images/faces_128_filter_samples.png" />

<img src="notebook-images/faces_arithmetic_collage_v2.png" />


Bu çalışmalar oldukça karmaşık olsa da, aslında GAN oluşturmak o kadar da zor değildir...
Örneğin MNIST üzerinden vanilla-GAN kullanılarak oluşan sentetik rakamlar aşağıdadır.


<img src="notebook-images/gan-animation.gif" />
Çalışma boyunca en stabil GAN mimarilerinden olan WGAN kullanarak CIFAR10 veri setinde yeni objeler oluşturacağız. 

## GAN Mimarisi
### Örnek olarak DCGAN (derin evrişimsel üretici ağ yapısı)

GAN yapısı iki modelin bir araya gelmesinden oluşur: 
Üretici model ve Ayırt edici model:
![caption](notebook-images/GAN_Overall.png)
Ayırt edici model elindeki resmin gerçek verisetindeki resimlere veya uydurma bir olabileceğini sınıflayan basit bir ikili sınıflandırma yapısıdır. Bilinen bir evrişimsel sinir ağı yapısında olabilir.

Üretici Model ise veri seti içinden rastgele girdi değerlerini alıp bunları yeni resimlere dönüştürür. Üretici ağ genelde evrişim işleminin tersini alarak çalışmaktadır ancak güncel mimarilerde bu da değişmektedir.

Üretici ve Ayırt Edici ağlarının ağırlık ve biasları backpropagation (geri besleme) algoritması ile birlikte hesaplanır.
Ayırt edici ağ, üretilen resimleri gerçek veya uydurma olarak ayırmaya çalışırken, üretici ağ ise ayırt edici ağı kandıracak resimleri ayırt edici ağı daha iyi kandıracak şekilde üretmeye çalışır.
![caption](notebook-images/ganm.png)
![caption](notebook-images/GAN Algorithm.png)

![caption](notebook-images/ganthing.png)
- Mavi çizgi Ayırt edici Ağ için loss fonksiyonu grafiği
- Siyah Nokta veri seti miktarı
- Yeşil çizgi üretilen veri seti miktarı


a) training başı
b) yeni resimler üretilmeye başladıkça
c) daha çok resim üretildiğindeki durum
d) girdi veriseti ile üretilen veriseti aynı boyutlara ulaştığındaki son durum ~ Nash-Equilibrium...

## Hadi başlayalım...

Bu uygulamada [Keras](https://keras.io/)  ve  [TensorFlow](https://www.tensorflow.org/), kullanağız.


## Veri seti:
### cifar 10 
<img src="notebook-images/cifar10.png" />


Kod  https://github.com/martinarjovsky/WassersteinGAN kaynağından uyarlanmıştır.

In [ ]:
## Keras arkasında tensorflow backend kullanıyoruz
import os
os.environ['KERAS_BACKEND']='tensorflow' 
os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,optimizer=fast_compile'

In [ ]:
import keras.backend as K
K.set_image_data_format('channels_first')
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)


## Ayırt edici ağ

WassersteinGAN-DCGAN'ın üzerinde farklı bir kayıp (loss) fonksiyonu kullanarak çalıştığı için ayırt edici ağ bir evrişimsel sinir ağıdır, cifar 10'un resim boyutu 32x32 olduğu için girdi olarak 32x32x3 kanallı resim girdisini alır, ve bunu veri seti içinde bulunan  "gerçek" veya üretici ağ tarafından oluşturulmuş "uydurma" olarak ikili sınıflamalardan birini temsil eden bir skalar değere dönüştürür.

![caption](notebook-images/GAN_D.png)
Ayırt edici ağ yapısı [TensorFlow'un CNN modeli](https://www.tensorflow.org/get_started/mnist/pros) gibi olsa da 3 evrişimsel sinir ağı ve 4x4 filtreden oluşmaktadır.

In [ ]:
def DCGAN_D(isize, nz, nc, ndf, n_extra_layers=0):
    assert isize%2==0
    _ = inputs = Input(shape=(nc, isize, isize))
    _ = Conv2D(filters=ndf, kernel_size=4, strides=2, use_bias=False,
                        padding = "same",
                        kernel_initializer = conv_init, 
                        name = 'initial.conv.{0}-{1}'.format(nc, ndf)             
                        ) (_)
    _ = LeakyReLU(alpha=0.2, name = 'initial.relu.{0}'.format(ndf))(_)
    csize, cndf = isize// 2, ndf
    while csize > 5:
        assert csize%2==0
        in_feat = cndf
        out_feat = cndf*2
        _ = Conv2D(filters=out_feat, kernel_size=4, strides=2, use_bias=False,
                        padding = "same",
                        kernel_initializer = conv_init,
                        name = 'pyramid.{0}-{1}.conv'.format(in_feat, out_feat)             
                        ) (_)
        if 0: # toggle batchnormalization
            _ = BatchNormalization(name = 'pyramid.{0}.batchnorm'.format(out_feat),                                   
                                   momentum=0.9, axis=1, epsilon=1.01e-5,
                                   gamma_initializer = gamma_init, 
                                  )(_, training=1)        
        _ = LeakyReLU(alpha=0.2, name = 'pyramid.{0}.relu'.format(out_feat))(_)
        csize, cndf = (csize+1)//2, cndf*2
    _ = Conv2D(filters=1, kernel_size=csize, strides=1, use_bias=False,
                        kernel_initializer = conv_init,
                        name = 'final.{0}-{1}.conv'.format(cndf, 1)         
                        ) (_)
    outputs = Flatten()(_)
    return Model(inputs=inputs, outputs=outputs)


## Üretici Ağ
Ayırt edici ağımızı tanımladığımıza göre üretici ağımızı tanımlayalım:

![caption](notebook-images/GAN_g.png)

Kernel boyutu 4x4 olan ve 3D evrişim modelinin transpose halini içeren üç adet ters evrişim işleminden oluşur. 
Sonuçta 32x32x3 resimler oluşturur.
Burada hiperbolik tanjant aktivasyon fonksiyonu kullanılmıştır. https://keras.io/activations/

<img src="notebook-images/TanhReal.gif" />


In [ ]:
def DCGAN_G(isize, nz, nc, ngf, n_extra_layers=0):
    cngf= ngf//2
    tisize = isize
    while tisize > 5:
        cngf = cngf * 2
        assert tisize%2==0
        tisize = tisize // 2
    _ = inputs = Input(shape=(nz,))
    _ = Reshape((nz, 1,1))(_)
    _ = Conv2DTranspose(filters=cngf, kernel_size=tisize, strides=1, use_bias=False,
                           kernel_initializer = conv_init, 
                           name = 'initial.{0}-{1}.convt'.format(nz, cngf))(_)
    _ = BatchNormalization(gamma_initializer = gamma_init, momentum=0.9, axis=1, epsilon=1.01e-5,
                               name = 'initial.{0}.batchnorm'.format(cngf))(_, training=1)
    _ = Activation("relu", name = 'initial.{0}.relu'.format(cngf))(_)
    csize, cndf = tisize, cngf
    

    while csize < isize//2:
        in_feat = cngf
        out_feat = cngf//2
        _ = Conv2DTranspose(filters=out_feat, kernel_size=4, strides=2, use_bias=False,
                        kernel_initializer = conv_init, padding="same",
                        name = 'pyramid.{0}-{1}.convt'.format(in_feat, out_feat)             
                        ) (_)
        _ = BatchNormalization(gamma_initializer = gamma_init, 
                                   momentum=0.9, axis=1, epsilon=1.01e-5,
                                   name = 'pyramid.{0}.batchnorm'.format(out_feat))(_, training=1)
        
        _ = Activation("relu", name = 'pyramid.{0}.relu'.format(out_feat))(_)
        csize, cngf = csize*2, cngf//2
    _ = Conv2DTranspose(filters=nc, kernel_size=4, strides=2, use_bias=False,
                        kernel_initializer = conv_init, padding="same",
                        name = 'final.{0}-{1}.convt'.format(cngf, nc)
                        )(_)
    outputs = Activation("tanh", name = 'final.{0}.tanh'.format(nc))(_)
    return Model(inputs=inputs, outputs=outputs)


Parametrelerimizi tanımlayalım.

In [ ]:
nc = 3 # number of channels
nz = 100
ngf = 64 # number of discriminator  features 
ndf = 64 # number of generator features
n_extra_layers = 0
Diters = 5 # iteration dimension
λ = 10 # wasserstein loss katsayısı

imageSize = 32
batchSize = 64
lrD = 1e-4 # ayırt edici ağ öğrenme katsayısı
lrG = 1e-4 # üretici ağ ağ öğrenme katsayısı


Ayırt edici ağ yapımız ve parametre sayıları:

In [ ]:
netD = DCGAN_D(imageSize, nz, nc, ndf, n_extra_layers)
netD.summary()

Üretici ağ yapımız ve parametre sayıları:

In [ ]:
netG = DCGAN_G(imageSize, nz, nc, ngf, n_extra_layers)
netG.summary()

Keras'tan optimize edici fonksiyonlarımızı çekelim:

In [ ]:
from keras.optimizers import RMSprop, SGD, Adam

Wasserstein Uzaklık fonksiyonumuzu tanımlayalım:
Bu uzaklık, normal olmayan sınırların ölçümünü kolaylaştırır...
p'nin 1'den büyük ve Pp(M) nin  M içindeki x0 için p. momentindeki µ'nün M'deki olasılık ölçütü olduğunu var sayarsak...
<img src="notebook-images/mm.png" />
µ ve υ arasındaki p. boyuttaki olasılık ölçütü:  <img src="notebook-images/pp.png" />

<img src="notebook-images/ww.png" />

<img src="notebook-images/w-distance.png" />

In [ ]:
netD_real_input = Input(shape=(nc, imageSize, imageSize))
noisev = Input(shape=(nz,))
netD_fake_input = netG(noisev)

ϵ_input = K.placeholder(shape=(None,1,1,1))
netD_mixed_input = Input(shape=(nc, imageSize, imageSize),
    tensor=ϵ_input * netD_real_input + (1-ϵ_input) * netD_fake_input)


loss_real = K.mean(netD(netD_real_input))
loss_fake = K.mean(netD(netD_fake_input))

grad_mixed = K.gradients(netD(netD_mixed_input), [netD_mixed_input])[0]
norm_grad_mixed = K.sqrt(K.sum(K.square(grad_mixed), axis=[1,2,3]))
grad_penalty = K.mean(K.square(norm_grad_mixed -1))

loss = loss_fake - loss_real + λ * grad_penalty


training_updates = Adam(lr=lrD, beta_1=0.0, beta_2=0.9).get_updates(netD.trainable_weights,[],loss)
netD_train = K.function([netD_real_input, noisev, ϵ_input],
                        [loss_real, loss_fake],    
                        training_updates)

In [ ]:
loss = -loss_fake 
training_updates = Adam(lr=lrG, beta_1=0.0, beta_2=0.9).get_updates(netG.trainable_weights,[], loss)
netG_train = K.function([noisev], [loss], training_updates)


## cifar 10 indirme ve hazırlık

In [ ]:
from PIL import Image
import numpy as np
import tarfile

# Download dataset
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
import os
import urllib
from urllib.request import urlretrieve
def reporthook(a,b,c):
    print("\rdownloading: %5.1f%%"%(a*b*100.0/c), end="")
tar_gz = "cifar-10-python.tar.gz"
if not os.path.isfile(tar_gz):
        print('Downloading data from %s' % url)
        urlretrieve(url, tar_gz, reporthook=reporthook)

import pickle
train_X=[]
train_y=[]
tar_gz = "cifar-10-python.tar.gz"
with tarfile.open(tar_gz) as tarf:
    for i in range(1, 6):
        dataset = "cifar-10-batches-py/data_batch_%d"%i
        print("load",dataset)
        with tarf.extractfile(dataset) as f:
            result = pickle.load(f, encoding='latin1')
        train_X.extend( result['data'].reshape(-1,3,32,32)/255*2-1)
        train_y.extend(result['labels'])
    train_X=np.float32(train_X)
    train_y=np.int32(train_y)
    dataset = "cifar-10-batches-py/test_batch"
    print("load",dataset)
    with tarf.extractfile(dataset) as f:
        result = pickle.load(f, encoding='latin1')
        test_X=np.float32(result['data'].reshape(-1,3,32,32)/255*2-1)
        test_y=np.int32(result['labels'])
        

In [ ]:
train_X = np.concatenate([train_X, test_X])
train_X = np.concatenate([train_X[:,:,:,::-1], train_X])

### Cifar 10 sınıflarını görselleştirme

In [ ]:
from IPython.display import display
def showX(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ( (X+1)/2*255).clip(0,255).astype('uint8')
    # N*3072 -> N*3*32*32 -> 32 * 32N * 3
    int_X = np.moveaxis(int_X.reshape(-1,3,32,32), 1, 3)
    int_X = int_X.reshape(rows, -1, 32, 32,3).swapaxes(1,2).reshape(rows*32,-1, 3)
    display(Image.fromarray(int_X))
showX(train_X[:20])
print(train_y[:20])
name_array = np.array("airplane car bird cat deer dog frog horse boat truck".split(' '))
print(name_array[train_y[:20]])

Gürültü ekliyoruz...

In [ ]:
fixed_noise = np.random.normal(size=(batchSize, nz)).astype('float32')

## Yeni resimler üretelim!

GAN yapısında dikkat etmemiz gereken nokta; kayıp (loss) fonksiyonlarımız. Elimizde iki adet loss fonksiyonu var, bir tanesi üretici ağın daha iyi resim oluşturmasını sağlıyor diğeri de aynı zamanda ayırt edici ağın üretilen resimlerle verisetinde bulunan resimleri ayırt etmesini sağlıyor.

Üretici ve ayırt edici ağlar birlikte çalıştırıldığından, ayırt edici ağ gerçek ve uydurma resimleri ayırt etmede daha iyi hale geliyor, bu sırada üretici ağ ise ağırlık ve bias değerlerini ikna edici resimler oluşturmak üzere ayarlıyor:

### İşte sonuçlar:

In [ ]:
import time
t0 = time.time()
niter = 100
gen_iterations = 0
errG = 0
targetD = np.float32([2]*batchSize+[-2]*batchSize)[:, None]
targetG = np.ones(batchSize, dtype=np.float32)[:, None]
for epoch in range(niter):
    i = 0
    np.random.shuffle(train_X)
    batches = train_X.shape[0]//batchSize
    while i < batches:
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            _Diters = 100
        else:
            _Diters = Diters
        j = 0
        while j < _Diters and i < batches:
            j+=1
            real_data = train_X[i*batchSize:(i+1)*batchSize]
            i+=1
            noise = np.random.normal(size=(batchSize, nz))        
            ϵ = np.random.uniform(size=(batchSize, 1, 1 ,1))        
            errD_real, errD_fake  = netD_train([real_data, noise, ϵ])
            errD = errD_real - errD_fake
       
        if gen_iterations%500==0:
            print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, niter, i, batches, gen_iterations,errD, errG, errD_real, errD_fake), time.time()-t0)
            fake = netG.predict(fixed_noise)
            showX(fake, 4)
        
        noise = np.random.normal(size=(batchSize, nz))        
        errG, = netG_train([noise])
        gen_iterations+=1 
        

## Çeşitli zorluklar ve Dikkat edilecek noktalar

GAN'lar çalıştırılması zor yapılardır, doğru hiperparametre ayarlaması, ağ yapısı, çalıştırılma protokolü gerektirler. Eğer hiperparametreler doğru ayarlanamazsa, ayırt edici ağ üretici ağı yener veya tam tersi olarak üretici ağ kazanır. Bu durumlarda doğru üretim gerçekleşmez.


Eğer ayırt edici ağ çok güçlü ayırt ederse, tüm üretilen resimleri her zaman uyduruk olarak sınıflar, bu durumda üretici ağ için türev hesaplanamaz ve dolayısı ile hiç üretim yapılamaz.


Bir diğer sorun da **modelin çökmesidir**. Üretici ağ ayırt edici ağ içinde bir zayıflık keşveder ve gürültüyü umursamadan hep aynı tip resimler oluşturur. Bu öğrenme katsayılarının iyi ayarlanması ile çözülebilir.


Gan'lar ile ilgili daha fazla ipucu için: ["GAN hacks"](https://github.com/soumith/ganhacks) sayfasını ziyaret edebilirsiniz.

## Diğer Kaynaklar

- [Orijinal GAN makalesi](https://arxiv.org/abs/1406.2661) by Ian Goodfellow et al 2014
- [En kapsamlı GAN makalesi ](https://arxiv.org/abs/1701.00160) Goodfellow
- [Alec Radford, Luke Metz, ve Soumith Chintala DCGAN makaleleri ](https://arxiv.org/abs/1511.06434) DCGAN'ın ilk tanıtım makalesi ve kodu [their DCGAN code on GitHub](https://github.com/Newmu/dcgan_code).
- [ Agustinus Kristiadi'den GAN refernasları](https://github.com/wiseodd/generative-models), tensorflow